# Explore the matrix multiplications
    Showcase a matmul operation
    Expand the use of matmal with scalars

In [1]:
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-26 21:16:01.210 | INFO     | SiliconDriver   - Detected 1 PCI device
2023-10-26 21:16:01.223 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-10-26 21:16:01.223 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-10-26 21:16:01.223 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-26 21:16:01.225 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaN

In [2]:
h = 32
w = 32 * 2

# Initialize tensors a and b with random values using torch

In [3]:
torch_a = torch.randn((h, w), dtype=torch.bfloat16)
torch_b = torch.randn((w, h), dtype=torch.bfloat16)

a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)

a = ttnn.to_device(a, device)
b = ttnn.to_device(b, device)

# Matrix multiply tensor a and b

In [4]:
output = a @ b

2023-10-26 21:16:04.078 | WARNING  | ttnn.core:reshape:468 - Given reshape operation could not be run on the TT device. Defaulting to torch implementation


# Print the result of the matrix multiply

In [5]:
print("Printing ttnn tensor")
print(output.shape)
print(output[:1])

Printing ttnn tensor
[32, 32]
Tensor([ [-6.375, -0.375, -4.15625, 7.15625, 0.0639648, 5.46875, 5.25, 7.25, 2.5625, -2.42188, 2.82812, -2.48438, 7.5, 9.4375, 9.625, -11.375, 4.71875, -4.125, -5.84375, 4.15625, -13.625, -2.625, -4.15625, -8.75, 10.1875, -0.294922, 4.875, 0.894531, -7.78125, -5.25, -10.4375, -4.40625]], dtype=bfloat16 )



# Matrix multiply tensor a and b and use more performant config

# Print the result of the matrix multiply

In [6]:
print("Printing ttnn tensor")
print(output.shape)
print(output[:1])

Printing ttnn tensor
[32, 32]
Tensor([ [-6.375, -0.375, -4.15625, 7.15625, 0.0639648, 5.46875, 5.25, 7.25, 2.5625, -2.42188, 2.82812, -2.48438, 7.5, 9.4375, 9.625, -11.375, 4.71875, -4.125, -5.84375, 4.15625, -13.625, -2.625, -4.15625, -8.75, 10.1875, -0.294922, 4.875, 0.894531, -7.78125, -5.25, -10.4375, -4.40625]], dtype=bfloat16 )



# Free tensor

In [7]:
ttnn.free(output)

# Free the device

In [8]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
